In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os, wandb
os.environ["WANDB_API_KEY"] = "29ef10a563126c993b31f306e5a17f55b237e430"
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ajagekarakshay (use `wandb login --relogin` to force relogin)


True

In [69]:
import numpy as np
import gym
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import conrl as crl
from conrl import  Q, Policy

In [70]:
env = gym.make('Pendulum-v1')
env.seed(2)

[2]

In [71]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.fc3(x))


class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size, seed, fcs1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fcs1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fcs1 = nn.Linear(state_size, fcs1_units)
        self.fc2 = nn.Linear(fcs1_units+action_size, fc2_units)
        self.fc3 = nn.Linear(fc2_units, 1)
        self.reset_parameters()

    def reset_parameters(self):
        self.fcs1.weight.data.uniform_(*hidden_init(self.fcs1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        xs = F.relu(self.fcs1(state))
        x = torch.cat((xs, action), dim=1)
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [72]:
logger_config = {
    "project":"pendulum"
}

env_mon = crl.wrapper.MonitorStats(env, 
                crl.logger.WandbRL, **logger_config)

episode/avg_reward,▇▆█▁▆▅▅
episode/avg_step_duration_ms,▁▁▁▁█▁▂
episode/episode,▁▂▃▅▆▇█
episode/return,▇▆█▁▆▅▅
episode/smooth_return,█▅▇▁▂▁▁
episode/steps,▁▁▁▁▁▁▁
episode/avg_reward,-6.0513
episode/avg_step_duration_ms,590.67408
episode/episode,7
episode/return,-1210.25975
episode/smooth_return,-1164.942


2021-10-27 13:57:51.116919: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-27 13:57:51.116976: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [73]:
import copy

actor = Actor(3, 1, seed=2)
critic = Critic(3,1, seed=2)

actor_copy = Actor(3, 1, seed=2)
actor_copy.load_state_dict(copy.deepcopy(actor.state_dict()))
critic_copy = Critic(3,1, seed=2)
critic_copy.load_state_dict(copy.deepcopy(critic.state_dict()))

# env_mon.logger.experiment.watch(actor, log="all")
# env_mon.logger.experiment.watch(actor_copy, log="all")
# env_mon.logger.experiment.watch(critic, log="all")
# env_mon.logger.experiment.watch(critic_copy, log="all")

<All keys matched successfully>

In [74]:
q = Q(env_mon, critic, torch.optim.Adam, optim_args=dict(lr=1E-3))
#q_targ = q.copy(deep=True)
q_targ = Q(env_mon, critic_copy, torch.optim.Adam, optim_args=dict(lr=1E-3))

pi = Policy(env_mon, model=actor, optimizer=torch.optim.Adam, optim_args=dict(lr=1E-4))
#pi_targ = pi.copy(deep=True) #deep=True)
pi_targ = Policy(env_mon, model=actor_copy, optimizer=torch.optim.Adam, optim_args=dict(lr=1E-4))

Torch policy instantiated
Torch policy instantiated


In [93]:
qpar = list(q.model.parameters())
ppar = list(pi.model.parameters())

qpar_targ = list(q_targ.model.parameters())
ppar_targ = list(pi_targ.model.parameters())

In [76]:
pi_noisy = crl.explore.OUprocess(pi, 1, 2)

In [77]:
tape = crl.recorder.NStepRecorder(n=1, gamma=0.99)
buffer = crl.replay.ReplayBuffer(capacity=100000)

qlearning = crl.objective.QLearningwithPolicy(q, 
                                            q_targ=q,
                                            pi_targ=pi_targ, 
                                            loss_function=F.mse_loss)

dpg = crl.objective.DeterministicPG(pi, q)

In [78]:
# Store params (0th)
idx = 0
# detach().numpy()
qstore = [qpar[idx].ravel()]
qtstore = [qpar_targ[idx].ravel()]
pstore = [ppar[idx].ravel()]
ptstore = [ppar_targ[idx].ravel()]

In [84]:
episodes = 1
for _ in tqdm(range(episodes), desc="Episode"):
    s = env_mon.reset()
    pi_noisy.reset()

    for t in range(300):       ############# changed
        a = pi_noisy.act(s)
        a = np.clip(a, -1, 1) ### added cliping
        s_next,r,done,_ = env_mon.step(a)

        tape.add(s,a,r,done,s_next)
        while tape:
            transition = tape.pop()
        buffer.add(transition)

        if done:
            break

        s = s_next

        if len(buffer) >= 128:
            #break
            transition_batch = buffer.sample(batch_size=128)
            
            qlearning.update(transition_batch)
            #qstore.append( qpar[idx].ravel() )
            #print(list(qlearning.q.parameters())[0])

            dpg.update(transition_batch)
            #pstore.append( ppar[idx].ravel() )
            #print(list(dpg.pi.parameters())[0])

            q_targ.soft_update(q, tau=1E-3)
            #qtstore.append( qpar_targ[idx].ravel() )
            print(list(q_targ.parameters())[0])

            pi_targ.soft_update(pi, tau=1E-3)
            #ptstore.append( ppar_targ[idx].ravel() )
            #print(list(pi_targ.parameters())[0])

Episode:   0%|          | 0/1 [00:00<?, ?it/s]

Episode: 5 | Timesteps: 200 |                    Avg_reward: -5.376752684125235 | Returns: -1075.3505368250471
Parameter containing:
tensor([[-0.0111,  0.1210,  0.0359],
        [-0.0365, -0.0025, -0.0072],
        [ 0.0172, -0.0017,  0.0155],
        ...,
        [-0.0043,  0.0394,  0.0133],
        [ 0.0370, -0.0471,  0.0454],
        [-0.0283,  0.0034,  0.0251]], requires_grad=True)
Parameter containing:
tensor([[-0.0110,  0.1212,  0.0359],
        [-0.0364, -0.0025, -0.0072],
        [ 0.0173, -0.0017,  0.0155],
        ...,
        [-0.0043,  0.0394,  0.0133],
        [ 0.0370, -0.0471,  0.0454],
        [-0.0283,  0.0034,  0.0251]], requires_grad=True)
Parameter containing:
tensor([[-0.0108,  0.1214,  0.0359],
        [-0.0363, -0.0025, -0.0071],
        [ 0.0173, -0.0016,  0.0154],
        ...,
        [-0.0042,  0.0394,  0.0133],
        [ 0.0370, -0.0471,  0.0454],
        [-0.0283,  0.0034,  0.0251]], requires_grad=True)
Parameter containing:
tensor([[-0.0106,  0.1217,  0.035

In [100]:
a = buffer.sample(21)

In [99]:
b = buffer.sample(21)

In [103]:
c = buffer.sample(21)

In [110]:
a.Gamma, b.Gamma, c.Gamma

(array([0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99,
        0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99]),
 array([0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99,
        0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99]),
 array([0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99,
        0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99]))

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/ajagekarakshay/pendulum/178tlwgp/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/ajagekarakshay/pendulum/178tlwgp/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/ajagekarakshay/pendulum/178tlwgp/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/ajagekarakshay/pendulum/178tlwgp/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Err

In [44]:
qstore[0], qstore[-1]

(tensor([ 0.0511,  0.0382, -0.0968,  ..., -0.0283,  0.0034,  0.0251],
        grad_fn=<AsStridedBackward>),
 tensor([ 0.0511,  0.0382, -0.0968,  ..., -0.0283,  0.0034,  0.0251],
        grad_fn=<ViewBackward>))

In [34]:
pstore[0], pstore[-1]

(tensor([-0.0091,  0.0113, -0.0216,  ...,  0.0313,  0.1073, -0.0872],
        grad_fn=<AsStridedBackward>),
 tensor([-0.0091,  0.0113, -0.0216,  ...,  0.0313,  0.1073, -0.0872],
        grad_fn=<ViewBackward>))

In [88]:
qlearning.q._setup(tb)

In [89]:
qlearning.flush()

In [90]:
loss = qlearning.compute_loss(tb)

In [91]:
qlearning.compute_grads(loss)

In [92]:
qlearning.q._apply_grads(None)

In [94]:
qpar[0], qpar_targ[0]

(Parameter containing:
 tensor([[ 0.0753,  0.4428,  0.0624],
         [ 0.0240,  0.0343,  0.0298],
         [-0.0158,  0.0938, -0.0115],
         ...,
         [ 0.0382,  0.0946,  0.0266],
         [ 0.0370, -0.0471,  0.0454],
         [-0.0283,  0.0034,  0.0251]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.0090,  0.1666,  0.0393],
         [-0.0254, -0.0035, -0.0013],
         [ 0.0132,  0.0113,  0.0116],
         ...,
         [ 0.0039,  0.0421,  0.0158],
         [ 0.0370, -0.0471,  0.0454],
         [-0.0283,  0.0034,  0.0251]], requires_grad=True))

In [95]:
q_targ.soft_update(q, 0.2)

In [96]:
qpar[0], qpar_targ[0]

(Parameter containing:
 tensor([[ 0.0753,  0.4428,  0.0624],
         [ 0.0240,  0.0343,  0.0298],
         [-0.0158,  0.0938, -0.0115],
         ...,
         [ 0.0382,  0.0946,  0.0266],
         [ 0.0370, -0.0471,  0.0454],
         [-0.0283,  0.0034,  0.0251]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.0223,  0.2218,  0.0439],
         [-0.0155,  0.0041,  0.0049],
         [ 0.0074,  0.0278,  0.0070],
         ...,
         [ 0.0108,  0.0526,  0.0179],
         [ 0.0370, -0.0471,  0.0454],
         [-0.0283,  0.0034,  0.0251]], requires_grad=True))

In [85]:
qlearning.update(tb)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


NameError: name 'tb' is not defined

In [ ]:
qpar[0], qpar[0].grad

In [ ]:
qpar[0], qpar[0].grad

In [ ]:
a = qstore[0]
for i in qstore:
    if a == i:
        print("True")

In [ ]:
ppar[0]

In [86]:
tb = buffer.sample(128)

In [ ]:
tb.S, tb.A, tb.R

In [ ]:
tb.S

In [87]:
qlearning.flush()

In [ ]:
qpar = list(q.model.parameters())
ppar = list(pi.model.parameters())

qpar_targ = list(q_targ.model.parameters())
ppar_targ = list(pi_targ.model.parameters())


In [ ]:
qpar[0], qpar[0].grad, qpar_targ[0], qpar_targ[0].grad

In [ ]:
qpar[0], qpar[0].grad, qpar_targ[0], qpar_targ[0].grad

In [ ]:
qlearning.update(tb)

In [ ]:
ppar[0], ppar[0].grad, ppar_targ[0], ppar_targ[0].grad

In [ ]:
qlearning.update(tb)

In [ ]:
qpar[0], qpar[0].grad, qpar_targ[0], qpar_targ[0].grad

In [ ]:
ppar[0], ppar[0].grad, ppar_targ[0], ppar_targ[0].grad

In [ ]:
dpg.update(tb)

In [ ]:
qpar[0], qpar[0].grad, qpar_targ[0], qpar_targ[0].grad

In [ ]:
ppar[0], ppar[0].grad, ppar_targ[0], ppar_targ[0].grad

In [ ]:
q_targ.soft_update(q, tau=0.2)

In [ ]:
qpar[0], qpar[0].grad, qpar_targ[0], qpar_targ[0].grad

In [ ]:
pi_targ.soft_update(pi, tau=0.2)

In [ ]:
ppar[0], ppar[0].grad, ppar_targ[0], ppar_targ[0].grad

In [ ]:
(qpar[0] == qpar_targ[0]).sum()

In [ ]:
qlearning.update(tb)

In [ ]:
qpar[0], qpar[0].grad

In [ ]:
dpg.flush()

In [ ]:
ppar[0], ppar[0].grad, qpar[0], qpar[0].grad

In [ ]:
ppar_targ[0], ppar_targ[0].grad, qpar_targ[0], qpar_targ[0].grad

In [ ]:
qpar_targ == qpar

In [ ]:
#Ap = dpg.pi(tb.S)
dpg.update(tb)

In [ ]:
dpg.flush()
loss = dpg.compute_loss(tb)

In [ ]:
loss 

In [ ]:
loss.backward()

In [ ]:
ppar[0], ppar[0].grad, qpar[0], qpar[0].grad

In [ ]:
pi.model.optimizer.step()

In [ ]:
ppar[0], ppar[0].grad, qpar[0], qpar[0].grad

In [ ]:
list(q.model.parameters())

In [ ]:
list(pi_noisy.pi.model.parameters()) 

In [ ]:
dpg.update(tb)

In [ ]:
k = list(pi_noisy.pi.model.parameters()) 

In [ ]:
list(pi.model.parameters()) == k

In [ ]:
dpg.flush()

In [ ]:
loss = dpg.compute_loss(tb)

In [ ]:
loss

In [ ]:
par = list(pi_noisy.pi.model.parameters())

In [ ]:
par[2].grad

In [ ]:
loss.backward()

In [ ]:
tb.S